In [ ]:
import pandas as pd
import os

### Creating a dataframe with my ratings and accom info
Data manually entered from my travel journal

In [ ]:
# Load existing data if it exists
csv_file = '../data/accommodations.csv'
if os.path.exists(csv_file):
    df = pd.read_csv(csv_file)
    accommodations = df.to_dict('records')
else:
    accommodations = []

while True:
    name = input("Accommodation name (or 'q' to quit): ")
    if name.lower() == 'q':
        break

    city = input("City: ")
    country = input("Country: ")
    rating = float(input("Your rating (0–5): "))
    comment = input("Comment: ")
    dates = input("Dates stayed (e.g., 'Jan 2020 - Mar 2020'): ")

    accommodations.append({
        "name": name,
        "city": city,
        "country": country,
        "my_rating": rating,
        "comment": comment,
        "dates_stayed": dates
    })
    
    # Save after each entry
    df = pd.DataFrame(accommodations)
    df.to_csv(csv_file, index=False)
    print(f"✓ Saved! Total entries: {len(accommodations)}")

### Looking through rows to check for any input errors in the dates:

In [ ]:
df = pd.read_csv("../data/accommodations.csv")
for row in df.index:
    print(f"{df.loc[row, 'name']}")
    print(f"{df.loc[row, 'dates_stayed']}\n")

### Creating an enriched dataset using Google Places API

In [ ]:
import requests

API_KEY = os.environ["GP_API_KEY"]

def get_place_data(query):
    url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    params = {
        "query": query,
        "key": API_KEY
    }

    response = requests.get(url, params=params)
    data = response.json()

    if data["results"]:
        result = data["results"][0]
        return {
            "name": result["name"],
            "lat": result["geometry"]["location"]["lat"],
            "lng": result["geometry"]["location"]["lng"],
            "google_rating": result.get("rating"),
            "review_count": result.get("user_ratings_total")
        }
    return None

Get coordinates and average rating for each accomodation

In [ ]:
# Enrich accommodations data with coordinates and Google ratings
# Load accommodations from CSV
df = pd.read_csv('../data/accommodations.csv')
accommodations = df.to_dict('records')

enriched_accommodations = []

for accommodation in accommodations:
    query = f"{accommodation['name']}, {accommodation['city']}, {accommodation['country']}"
    place_data = get_place_data(query)
    
    if place_data:
        enriched_row = {
            **accommodation,
            "latitude": place_data["lat"],
            "longitude": place_data["lng"],
            "google_rating": place_data["google_rating"],
            "review_count": place_data["review_count"]
        }
    else:
        enriched_row = {
            **accommodation,
            "latitude": None,
            "longitude": None,
            "google_rating": None,
            "review_count": None
        }
    
    enriched_accommodations.append(enriched_row)
    print(f"Extracted {accommodation['name']} location data")

# Create new dataframe with enriched data
df_enriched = pd.DataFrame(enriched_accommodations)
print(f"\nEnriched dataframe created with {len(df_enriched)} accommodations")
print(df_enriched.head())

In [ ]:
df_enriched.to_csv('../data/accommodations_enriched.csv', index=False)